<a href="https://colab.research.google.com/github/GrigoreGeorgeAlexandru/Colab-projects/blob/main/Laborator7_F_Titanic_Log_Regression_stud.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Regresie logistică - exemplu pe dataset-ul Titanic


In [ ]:
!pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.0/317.0 MB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.1-py2.py3-none-any.whl size=317488491 sha256=d35d3151f88969f73ac4659280cfd38772df0c3cc3dbb2a498454148c5a4ecdb
  Stored in directory: /root/.cache/pip/wheels/80/1d/60/2c256ed38dddce2fdd93be545214a63e02fbd8d74fb0b7f3a6
Successfully built pyspark


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from pyspark.sql import SparkSession

In [ ]:
spark = SparkSession.builder.appName('reglog_Titanic').getOrCreate()

In [ ]:
data = spark.read.csv('/content/drive/MyDrive/Data/titanic.csv',inferSchema=True,header=True)

In [ ]:
#Afisati schema datelor
data.printSchema()

root
 |-- PassengerId: integer (nullable = true)
 |-- Survived: integer (nullable = true)
 |-- Pclass: integer (nullable = true)
 |-- Name: string (nullable = true)
 |-- Gender: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- SibSp: integer (nullable = true)
 |-- Parch: integer (nullable = true)
 |-- Ticket: string (nullable = true)
 |-- Fare: double (nullable = true)
 |-- Cabin: string (nullable = true)
 |-- Embarked: string (nullable = true)



In [ ]:
#Afisati coloanele
data.columns

['PassengerId',
 'Survived',
 'Pclass',
 'Name',
 'Gender',
 'Age',
 'SibSp',
 'Parch',
 'Ticket',
 'Fare',
 'Cabin',
 'Embarked']

In [ ]:
#Selectati coloanele 'Survived',  'Pclass',  'Gender',  'Age',  'SibSp',  'Parch',  'Fare',  'Embarked'
my_cols = data.select(['Survived',  'Pclass',  'Gender',  'Age',  'SibSp',  'Parch',  'Fare',  'Embarked'])

In [ ]:
#Stergeti randurile ce contin valori null
my_final_data = my_cols.na.drop()

### Lucrul cu coloane categoriale


In [ ]:
from pyspark.ml.feature import (VectorAssembler,VectorIndexer,
                                OneHotEncoder,StringIndexer)

In [ ]:
gender_indexer = StringIndexer(inputCol='Gender',outputCol='GenderIndex')
gender_encoder = OneHotEncoder(inputCol='GenderIndex',outputCol='GenderVec')

In [ ]:
embark_indexer = StringIndexer(inputCol='Embarked',outputCol='EmbarkIndex')
embark_encoder = OneHotEncoder(inputCol='EmbarkIndex',outputCol='EmbarkVec')

In [ ]:
#Asamblati coloanele 'Pclass',  'GenderVec',  'Age',  'SibSp',  'Parch',  'Fare',  'EmbarkVec' in vectorul "features"
assembler = VectorAssembler(inputCols=['Pclass',  'GenderVec',  'Age',  'SibSp',  'Parch',  'Fare',  'EmbarkVec'],outputCol='features')

## Pipeline


In [ ]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml import Pipeline

In [ ]:
log_reg_titanic = LogisticRegression(featuresCol='features',labelCol='Survived')

NameError: name 'LogisticRegression' is not defined

In [ ]:
#Inlantuiti gender_indexer,embark_indexer, gender_encoder,embark_encoder,assembler,log_reg_titanic ca etape in cadrul unui Pipeline
pipeline =Pipeline(stages=[gender_indexer,embark_indexer, gender_encoder,embark_encoder,assembler,log_reg_titanic])

In [ ]:
train_titanic_data, test_titanic_data = my_final_data.randomSplit([0.7,.3])

In [ ]:
#Antrenare pipeline
fit_model = pipeline.fit(train_titanic_data)

In [ ]:
#Aplicare model pe datele de test
results = fit_model.transform(test_titanic_data)

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator

In [ ]:
#evaluare
my_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction',
                                       labelCol='Survived')

In [ ]:
results.select('Survived','prediction').show()

+--------+----------+
|Survived|prediction|
+--------+----------+
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       0.0|
|       0|       1.0|
|       0|       1.0|
|       0|       0.0|
+--------+----------+
only showing top 20 rows



In [ ]:
AUC = my_eval.evaluate(results)

In [ ]:
AUC

0.7611955420466058